In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['op_setting_1', 'op_setting_2', 'op_setting_3']
sensor_names = ['sensor_measurement{}'.format(i+1) for i in range(0,21)]
col_names = index_names + setting_names + sensor_names

In [3]:
train1 = pd.read_csv('train_FD001.txt',sep=' ',header=None,index_col=False,names=col_names)
test1 = pd.read_csv('test_FD001.txt',sep=' ',header=None,index_col=False,names=col_names)

C:\Users\miqba\AppData\Local\Temp\ipykernel_38968\99324521.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  train1 = pd.read_csv('train_FD001.txt',sep=' ',header=None,index_col=False,names=col_names)
C:\Users\miqba\AppData\Local\Temp\ipykernel_38968\99324521.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  test1 = pd.read_csv('test_FD001.txt',sep=' ',header=None,index_col=False,names=col_names)


In [4]:
def categorize_zone(time_cycles, max_time_cycle):
    time_percent = time_cycles / max_time_cycle
    if time_percent <= 0.5:
        return "Safe Zone"
    elif time_percent <= 0.75:
        return "Moderate Zone"
    elif time_percent <= 1:
        return "Dangerous Zone"
    else:
        return "Failure"

def add_zones_rul_column(df):
    train_grouped_by_unit = df.groupby(by='unit_number') 
    max_time_cycles = train_grouped_by_unit['time_cycles'].max() 
    merged = df.merge(max_time_cycles.to_frame(name='max_time_cycle'), left_on='unit_number',right_index=True)
    
    merged["RUL"] = merged["max_time_cycle"] - merged['time_cycles']
    
    merged["zone"] = merged.apply(lambda row: categorize_zone(row['time_cycles'], row['max_time_cycle']), axis=1)
    
    merged = merged.drop("max_time_cycle", axis=1) 
    return merged

In [5]:
train1 = add_zones_rul_column(train1)
train1

,unit_number,time_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_measurement1,sensor_measurement2,sensor_measurement3,sensor_measurement4,sensor_measurement5,...,sensor_measurement14,sensor_measurement15,sensor_measurement16,sensor_measurement17,sensor_measurement18,sensor_measurement19,sensor_measurement20,sensor_measurement21,RUL,zone
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,Safe Zone
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,Safe Zone
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,Safe Zone
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,Safe Zone
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,Safe Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735,4,Dangerous Zone
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594,3,Dangerous Zone
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333,2,Dangerous Zone
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640,1,Dangerous Zone


In [6]:
X = train1.drop(columns=['RUL','zone'],axis=1)
y = train1['RUL']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
from sklearn.ensemble import RandomForestRegressor

In [9]:
model = RandomForestRegressor()
model.fit(X_train,y_train)
y_pred = model.predict(test1)
test1['Predicted_RUL'] = y_pred

In [17]:
test1[test1['unit_number']==5] [['unit_number','time_cycles','Predicted_RUL']].head()

,unit_number,time_cycles,Predicted_RUL
312,5,1,185.13
313,5,2,183.68
314,5,3,184.88
315,5,4,184.23
316,5,5,191.41


In [18]:
train1[train1['unit_number']==5] [['unit_number','time_cycles','RUL']].head()

,unit_number,time_cycles,RUL
847,5,1,268
848,5,2,267
849,5,3,266
850,5,4,265
851,5,5,264
